This notebook implements regression on the first round results with cross validation.

## Pipeline

- Data pre-processing: run codes/data_generating.py
- Reading data: read data/firstRound_4h.csv into pandas dataframe
- Cross validation: training (training, validating); testing, KFold (K = ?)
    - Emdedding
        - Onehot
        - Label 
    - Kernel
        - RBF
        - DotProduct
        - Spectrum 
            - lmer: l = ?
            - Padding_flag: add special characters before and after sequences, e.g. 'ACTGAA' -> 'ZZ' + 'ACTGAA' + 'ZZ'
            - gap_flag:add gapped features, e.g. 3-mer-1-gap
            - normalised_kernel: e.g. zero-mean, unit-norm, unit-var
        - Sum of Spectrum
            - a K_A + b K_B + c K_C, where a + b + c = 1
    - Regression model
        - Gaussian process regression 
            - alpha: scalar value add to diagonal 
            - heteroscedastic: noises are learned as well (same as normalise each replicates to have same derivatives)
- Evaluation
   - metric: e.g. Mean square error; R2
   - true label: either sample or mean of sample. 

## Splitting in terms of sequences vs samples?

For each sequence, we have at least three biological replicates. There are two ways splitting data into training and testing data: splitting in terms of sequences, where if one sequence is split into the training dataset, then all replicates of that sequences belong to the training dataset; splitting in terms of samples, where the replicates of one sequence can be in the training or testing dataset. The two methods both make sense in terms of the evaluation of Gaussian process regression.

Considering the goal is to design good sequences for the second round experiment, we expect for training, we do not have any information about the sequences in the testing dataset. That is, splitting in terms of sequences can better simulate the sequence design task. 

In this notebook, we show both two methods of splitting data. We expect that splitting in terms of samples provide a lower error in testing data since the model may have seen the sequences. The test error of splitting sequences should have higher variance, since the prediction depends on whether the sequences in test dataset similar to the sequences in the training dataset. But again, our goal is to decrease the test error as well as the variance for the case of splitting sequences. 

## Evaluate on samples vs sample mean?

For training, we use the samples as labels, since we model the label for each sequence as samples from a unknown (Gaussian) reward distribution.

For testing, we use the sample mean for the sequence (i.e. mean value of the three replicates) as label. The ideal label should be the true mean of underlying distribution (we assume is Gaussian) of a sequence, however, we do not know the true mean. The only choice is to use the sample mean to approximate the true mean, which has the risk the sample mean would deviate a lot from the true mean since the number of samples is quite low (and variance of samples is not low). 

## Choices to make within cross validation 

- alpha: scalar value add to diagonal 
- kernel
- lmer: l = ?
- For Sum_Spectrum_Kernel: b for (1 - b)/2 K_A + b K_B + (1-b)/2 K_C

In [1]:
# direct to proper path
import os
import sys
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
from collections import defaultdict
import math
import json
import xarray as xr

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import PairwiseKernel, DotProduct, RBF 
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import r2_score, mean_squared_error, make_scorer
from sklearn.model_selection import KFold

from codes.embedding import Embedding
from codes.environment import Rewards_env
from codes.ucb import GPUCB, Random
from codes.evaluations import evaluate, plot_eva
from codes.regression import *
from codes.kernels_for_GPK import Spectrum_Kernel, Sum_Spectrum_Kernel, WeightedDegree_Kernel

from ipywidgets import IntProgress
from IPython.display import display
import warnings
%matplotlib inline

/home/mengyan/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
kernel_dict = {
    'Spectrum_Kernel': Spectrum_Kernel,
    'Mixed_Spectrum_Kernel': Mixed_Spectrum_Kernel,
    'WD_Kernel': WeightedDegree_Kernel,
    'Sum_Spectrum_Kernel': Sum_Spectrum_Kernel,
    'WD_Kernel_Shift': WD_Shift_Kernel
    
}

In [3]:
# setting

cv = 5
test_size = 0.2
random_state = 24
embedding = 'label'
eva_on_ave_flag = True # true label is the sample mean instead of individual samples, since the prediction is the posterior mean
eva_metric = mean_squared_error # mean square error returns a more stable optimal hyparameter choice than r2 score

kernel_list = ['Spectrum_Kernel', 
               #'Mixed_Spectrum_Kernel',
               #'Sum_Spectrum_Kernel', 
               #'WD_Kernel',
               #'WD_Kernel_Shift'
              ]
alpha_list = [0.1, 1, 2, 3, 5]
l_lists = [[3], [6]]
b_list = [0.34, 0.6, 0.8]

weight_flag = False
padding_flag = False
gap_flag = False

## Cross Validation for sequences

In [4]:
Path = '../../data/firstRound_4h_normTrue_formatSeq.csv'

df = pd.read_csv(Path)
df.head(20)

,Unnamed: 0,Name,Group,RBS,RBS6,Rep1,Rep2,Rep3,Rep4,Rep5,AVERAGE,STD
0,0,RBS_1by1_0,reference,TTTAAGAAGGAGATATACAT,AGGAGA,NaN,2.783529,NaN,3.260245,3.225496,3.089757,0.265769
1,1,RBS_1by1_1,bps_noncore,CTTAAGAAGGAGATATACAT,AGGAGA,NaN,1.678119,NaN,1.589655,2.374458,1.880744,0.429851
2,2,RBS_1by1_2,bps_noncore,GTTAAGAAGGAGATATACAT,AGGAGA,NaN,0.670263,NaN,0.091788,0.295688,0.352580,0.293404
3,3,RBS_1by1_3,bps_noncore,ATTAAGAAGGAGATATACAT,AGGAGA,NaN,1.948917,NaN,1.299476,1.653915,1.634102,0.325173
4,4,RBS_1by1_4,bps_noncore,TCTAAGAAGGAGATATACAT,AGGAGA,NaN,2.201733,NaN,1.786760,1.687616,1.892036,0.272748
5,5,RBS_1by1_5,bps_noncore,TGTAAGAAGGAGATATACAT,AGGAGA,NaN,0.055417,NaN,-0.188262,0.006650,-0.042065,0.128937
6,6,RBS_1by1_6,bps_noncore,TATAAGAAGGAGATATACAT,AGGAGA,NaN,1.259399,NaN,0.539809,1.048676,0.949295,0.369946
7,8,RBS_1by1_8,bps_noncore,TTGAAGAAGGAGATATACAT,AGGAGA,NaN,1.689308,NaN,1.861887,1.713073,1.754756,0.093536
8,9,RBS_1by1_9,bps_noncore,TTAAAGAAGGAGATATACAT,AGGAGA,NaN,0.521988,NaN,0.061684,0.082107,0.221926,0.260062
9,10,RBS_1by1_10,bps_noncore,TTTCAGAAGGAGATATACAT,AGGAGA,NaN,0.858082,NaN,1.263810,1.032268,1.051387,0.203539


In [5]:
#df = df[df['Group'] != 'bps'].reset_index()
df.shape

(150, 12)

In [16]:
num_split = 5
num_repeat = 10
s_list = [0,1]
kernel = 'WD_Kernel_Shift'
alpha_list= [1e-10, 1e-5, 1e-4, 1e-3, 1e-2, 0.1, 0.5, 1, 5, 10, 20, 25, 30, 35, 40, 45, 50]
l_lists = [[3], [4], [5], [6]]

result_DataArray = Repeated_kfold(df, num_split, num_repeat, kernel, alpha_list, embedding, eva_metric, eva_on_ave_flag,
                                      l_lists, s_list)

In [21]:
import pickle

with open('repeated_kfold_wd_shift.pickle', 'wb') as handle:
    pickle.dump(result_DataArray, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('repeated_kfold_wd_shift.pickle', 'rb') as handle:
    result_pkl = pickle.load(handle)

In [22]:
result_pkl.loc[dict(train_test = 'Train')]

<xarray.DataArray (alpha: 17, l: 4, s: 2, num_repeat: 10, num_split: 5)>
array([[[[[2.54143051e-08, 1.90803156e-08, 1.21157716e-02,
           2.68249622e-08, 2.02513474e-08],
          [1.96137816e-08, 1.21302142e-02, 2.09172150e-08,
           1.88324099e-08, 1.21093537e-02],
          [1.54977317e-08, 8.08212709e-03, 1.79558586e-08,
           2.33082052e-08, 1.21222140e-02],
          ...,
          [1.21062796e-02, 2.94427354e-08, 2.02144118e-02,
           2.46399328e-08, 1.75105429e-08],
          [1.21157768e-02, 1.21024308e-02, 2.79308436e-08,
           8.08760099e-03, 1.26465677e-08],
          [1.21133204e-02, 1.21071887e-02, 2.66002221e-08,
           8.22110420e-08, 2.03956878e-08]],

         [[1.21059687e-02, 1.21179429e-02, 4.44203950e-08,
           2.44671441e-08, 4.89328580e-08],
          [8.08850764e-03, 2.17816003e-08, 1.21249691e-02,
           2.01945995e-02, 1.52268805e-08],
          [1.58561988e-08, 2.02073229e-02, 1.21292708e-02,
           1.21044292e-02, 1.99361937e-08],
...
          [5.50590572e-01, 4.94912116e-01, 4.85026504e-01,
           4.36992399e-01, 5.65334318e-01],
          [3.90850532e-01, 5.42888549e-01, 5.44664048e-01,
           5.26530842e-01, 5.01367847e-01],
          [4.59740826e-01, 4.59380508e-01, 4.63206421e-01,
           5.46623483e-01, 5.63289421e-01]],

         [[5.34303275e-01, 5.43707507e-01, 5.13406764e-01,
           4.57665742e-01, 4.51684509e-01],
          [4.79646311e-01, 4.27824093e-01, 5.19505465e-01,
           5.10685448e-01, 5.61740858e-01],
          [5.50748125e-01, 4.20157773e-01, 4.93447580e-01,
           5.07933692e-01, 5.29121999e-01],
          ...,
          [4.31513985e-01, 5.28793892e-01, 5.28346304e-01,
           4.99824761e-01, 5.02055334e-01],
          [4.69994653e-01, 4.89114992e-01, 5.11574998e-01,
           5.21254460e-01, 4.98495088e-01],
          [5.30203515e-01, 4.75078533e-01, 4.62958913e-01,
           5.03483124e-01, 5.34869265e-01]]]]])
Coordinates:
    train_test  <U5 'Train'
  * alpha       (alpha) float64 1e-10 1e-05 0.0001 0.001 ... 35.0 40.0 45.0 50.0
  * l           (l) <U3 '[3]' '[4]' '[5]' '[6]'
  * s           (s) int64 0 1
  * num_repeat  (num_repeat) int64 0 1 2 3 4 5 6 7 8 9
  * num_split   (num_split) int64 0 1 2 3 4
Attributes:
    eva_metric:      <function mean_squared_error at 0x7f552ad69c80>
    eva_on_average:  True

In [19]:
result_DataArray[1].mean(axis = -1).mean(axis = -1)

<xarray.DataArray (alpha: 17, l: 4, s: 2)>
array([[[8.21439744, 7.4126444 ],
        [5.3178352 , 5.08392929],
        [4.49050921, 4.51651294],
        [3.26777231, 3.30151216]],

       [[7.80574625, 7.56868442],
        [4.99489461, 5.28743465],
        [4.50721843, 4.52640975],
        [3.57734881, 3.34753943]],

       [[8.85792501, 7.53461966],
        [4.55834506, 5.17776118],
        [4.53240155, 4.456158  ],
        [3.19779175, 3.59134282]],

       [[7.87963601, 7.54334026],
        [4.83553552, 5.35211855],
        [4.53894184, 4.41904583],
        [3.34020126, 3.38115116]],

...

       [[0.85414432, 0.84237623],
        [0.83579311, 0.84201537],
        [0.80768796, 0.81247728],
        [0.80596954, 0.80971965]],

       [[0.84288352, 0.82116452],
        [0.82480723, 0.83770701],
        [0.81002831, 0.81565878],
        [0.80621415, 0.80968709]],

       [[0.82882071, 0.83324815],
        [0.81015206, 0.82916771],
        [0.81558496, 0.79344661],
        [0.80292341, 0.7932035 ]],

       [[0.82919608, 0.82676135],
        [0.80804288, 0.81385961],
        [0.80999258, 0.78546087],
        [0.78240262, 0.79291182]]])
Coordinates:
    train_test  <U5 'Test'
  * alpha       (alpha) float64 1e-10 1e-05 0.0001 0.001 ... 35.0 40.0 45.0 50.0
  * l           (l) <U3 '[3]' '[4]' '[5]' '[6]'
  * s           (s) int64 0 1

In [ ]:
optimal_para, test_scores = cross_val(df, cv, random_state, test_size, kernel_list,  alpha_list, embedding, eva_metric, eva_on_ave_flag,
                                      l_lists, b_list, weight_flag, padding_flag, gap_flag)

## Optimal Choices for splitting sequences case

- optimal kernel:  Spectrum_Kernel , 
- optimal alpha:  2 , 
- optiaml l list:  [2, 3, 4, 5, 6]

## Cross Validation for samples

In [8]:
Path = '../../data/firstRound_4h_normTrue_formatSample.csv'

df_samples = pd.read_csv(Path)
df_samples.head()

,Unnamed: 0,Name,RBS,RBS6,AVERAGE,STD,Group,variable,label
0,0,RBS_1by1_0,TTTAAGAAGGAGATATACAT,AGGAGA,3.089757,0.265769,reference,Rep2,2.783529
1,1,RBS_1by1_1,CTTAAGAAGGAGATATACAT,AGGAGA,1.880744,0.429851,bps_noncore,Rep2,1.678119
2,2,RBS_1by1_2,GTTAAGAAGGAGATATACAT,AGGAGA,0.352580,0.293404,bps_noncore,Rep2,0.670263
3,3,RBS_1by1_3,ATTAAGAAGGAGATATACAT,AGGAGA,1.634102,0.325173,bps_noncore,Rep2,1.948917
4,4,RBS_1by1_4,TCTAAGAAGGAGATATACAT,AGGAGA,1.892036,0.272748,bps_noncore,Rep2,2.201733


In [9]:
df_samples_frr = df_samples[df_samples['Group'] != 'Baseline data'].reindex()

In [10]:
df_samples_frr.shape

(450, 9)

In [ ]:
optimal_para, test_scores = cross_val(df_samples, cv, random_state, test_size, kernel_list, alpha_list, embedding, eva_metric, eva_on_ave_flag, 
                                      l_lists, b_list, weight_flag, padding_flag, gap_flag)

In [ ]:
optimal_para, test_scores = cross_val(df_samples_frr, cv, random_state, test_size, kernel_list, alpha_list, embedding, eva_metric, eva_on_ave_flag, 
                                      l_lists, b_list, weight_flag, padding_flag, gap_flag)

## Optimal Choices for splitting samples case

- optimal kernel:  Spectrum_Kernel , 
- optimal alpha:  0.1 , 
- optiaml l list:  [2, 3, 4, 5, 6]
    
As we expected, when splitting in terms of samples, we have a smaller test error and smaller test error variance as well compared splitting in terms of sequences.

## Summary

To summarise, for the second round design, we choose to use all first round results (NO baseline data), with 
- Spectrum kennel
- alpha = 2
- lmer, l = [2,3,4,5,6]
- with replicates normalisation
- Sample mean as true label
- normalised kernel
- (TODO) features: no gap, np padding, no weight?